# 구글 드라이브 mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

In [ ]:
%cd /content/drive/MyDrive/딥러닝/최종코드/DATASET

# import

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pylab as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

import os 
import io
import zipfile
from zipfile import ZipFile
import requests
import tensorflow as tf
import cv2
import json
import matplotlib.image as img
from PIL import Image

from tqdm import tqdm
import time

## seed 고정

In [ ]:
def my_seed_everywhere(seed: int = 0):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os

my_seed = 0
my_seed_everywhere(my_seed)

## 폴더 생성 함수 정의

In [ ]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

# 음식 resize

In [ ]:
%cd /content/drive/MyDrive/딥러닝/최종코드

In [ ]:
path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/food_zip' # 원본 폴더 안에 음식 종류별 zip파일로 이미지 존재 [용량이 커서 같이 포함시키지 못함]
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

In [ ]:
for zip_name in file_list[0:]:
    print(zip_name)
    folder_name = zip_name.split('.')[0]
    f_name = zip_name.split('.')[0]
    
    with ZipFile(data_path+f'{zip_name}', 'r') as zipObj:
        listOfFileNames = [file for file in zipObj.namelist() if file.endswith(".jpg") or file.endswith(".JPG")]
        
        fol_lst = []
        for i in listOfFileNames:
            fol_lst.append(i.split('/')[1])
        # 폴더 생성
        for i in fol_lst:
            createDirectory('/content/drive/MyDrive/딥러닝/최종코드/DATASET/'+f'음식원본/color_image')
            createDirectory('/content/drive/MyDrive/딥러닝/최종코드/DATASET/'+f'음식원본/gray_image')

        for img_num in tqdm(range(0,len(listOfFileNames))):
            image_name = listOfFileNames[img_num].split('.')[0] # 경로 포함한 이미지 이름
            img_real_name = listOfFileNames[img_num].split('/')[-1].split('.')[0] # 이미지 자체의 이름
            
            name = listOfFileNames[img_num]
            readed = zipObj.open(name) # binary 형태로 읽기

            src=Image.open(readed)        
            src = np.array(src)
            
            src_re=cv2.resize(src, (224,224))
            
            # 이미지 저장하기
            src_re = Image.fromarray(src_re) # NumPy array to PIL image
            src_re.save('/content/drive/MyDrive/딥러닝/최종코드/DATASET/'+f"음식원본/color_image/"+f'{f_name}_{img_real_name}_color.png')
            # 이미지 저장하기
            src_gray = src_re.convert('L') # 흑백 변환
            src_gray.save('/content/drive/MyDrive/딥러닝/최종코드/DATASET/'+f"음식원본/gray_image/"+f'{f_name}_{img_real_name}_gray.png')

# train, test 분리

In [ ]:
color_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/음식원본/color_image'
gray_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/음식원본/gray_image'

train_color_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/train/color_image/'
train_gray_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/train/gray_image/'
test_color_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/test/color_image/'
test_gray_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/test/gray_image/'

createDirectory(train_color_img_path)
createDirectory(train_gray_img_path)
createDirectory(test_color_img_path)
createDirectory(test_gray_img_path)

In [ ]:
num_lst=[]
for i in tqdm(file_list):
    num_lst.append(i.split('_')[2])
num_lst = list(set(num_lst))

for i in num_lst:
    lst=[]
    for j in tqdm(file_list):
        if j.split('_')[2] == i:
            name = '_'.join(j.split('.')[0].split('_')[:-1])
            lst.append(name)

    count = int(len(lst)*0.7) # 각 음식 폴더별로 갯수 구한 뒤 이미지 갯수의 0.7배는 train, 0.3배는 test로 들어가도록 함
    
    tr_lst = lst[:count]
    te_lst = lst[count:]

    for k in tqdm(tr_lst):
        color_name = color_img_path+'/'+k+'_color.png'
        gray_name = gray_img_path+'/'+k+'_gray.png'

        c_src=Image.open(color_name)
        g_src=Image.open(gray_name)
        # 이미지 저장하기
        c_src.save(train_color_img_path + k+'_color.png')
        g_src.save(train_gray_img_path + k+'_gray.png')

    for k in tqdm(te_lst):
        color_name = color_img_path+'/'+k+'_color.png'
        gray_name = gray_img_path+'/'+k+'_gray.png'

        c_src=Image.open(color_name)
        g_src=Image.open(gray_name)
        # 이미지 저장하기
        c_src.save(test_color_img_path + k+'_color.png')
        g_src.save(test_gray_img_path + k+'_gray.png')

### 추후 데이터셋을 줄이기 위해서 train은 절반, test는 4분의1 새로 저장

In [ ]:
tr_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/train/color_image2/'
te_img_path = '/content/drive/MyDrive/딥러닝/최종코드/DATASET/test/color_image2/'

createDirectory(tr_img_path)
createDirectory(te_img_path)

In [ ]:
file_list = os.listdir(train_color_img_path)

In [ ]:
for i in range(len(file_list)):
    if i % 2 == 0:
        color_name = train_color_img_path+file_list[i]
        c_src=Image.open(color_name)
        c_src.save(tr_img_path+file_list[i])

In [ ]:
test_list = os.listdir(test_color_img_path)

In [ ]:
for i in range(len(test_list)):
    if i % 4 == 0:
        color_name = test_color_img_path+test_list[i]
        c_src=Image.open(color_name)
        c_src.save(te_img_path+test_list[i])